#### Group Information

Group No: 

- Member 1: Angeline Teoh Qee (159023)
- Member 2: LeeJu Yi (158713)
- Member 3: Lee Ter Qin (159389)
- Member 4: Jee Rou Yi (159273)

### Project Description
The dataset used in this project is dermatology. The differential diagnosis of erythemato-squamous diseases is a real problem in dermatology. They all share the clinical features of erythema and scaling, with very little differences. The predictive models that we have chosen to perform multiclass classification are **K-Nearest Neighbors (KNN)** and **Neural Network**.


#### Import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error, mean_squared_error, r2_score

%config Completer.use_jedi = False

import warnings
warnings.filterwarnings('ignore')

#### Load the dataset

In [2]:
dataset = pd.read_csv("C:/Users/Angeline/OneDrive/Desktop/Year2/Y2S2/CPC251/Project/Dermatology/dermatology.csv")

Next, to understand the dataset, we will be printing information about the dataframe, such as number of columns, column labels, column data types, memory usage, range index, and the number of cells in each column (non-null values).

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   2       365 non-null    int64 
 1   2.1     365 non-null    int64 
 2   0       365 non-null    int64 
 3   3       365 non-null    int64 
 4   0.1     365 non-null    int64 
 5   0.2     365 non-null    int64 
 6   0.3     365 non-null    int64 
 7   0.4     365 non-null    int64 
 8   1       365 non-null    int64 
 9   0.5     365 non-null    int64 
 10  0.6     365 non-null    int64 
 11  0.7     365 non-null    int64 
 12  0.8     365 non-null    int64 
 13  0.9     365 non-null    int64 
 14  0.10    365 non-null    int64 
 15  3.1     365 non-null    int64 
 16  2.2     365 non-null    int64 
 17  0.11    365 non-null    int64 
 18  0.12    365 non-null    int64 
 19  0.13    365 non-null    int64 
 20  0.14    365 non-null    int64 
 21  0.15    365 non-null    int64 
 22  0.16    365 non-null    in

Since there are no label for each column, so we add column name for them

In [4]:
col_name=['erythema', 'scaling', 'definite_borders', 'itching',
       'koebner_phenomenon', 'polygonal_papules', 'follicular_papules',
       'oral_mucosal_involvement', 'knee_and_elbow_involvement',
       'scalp_involvement', 'family_history', 'melanin_incontinence',
       'eosinophils_in_the_infiltrate', 'pnl_infiltrate',
       'fibrosis_of_the_papillary_dermis', 'exocytosis', 'acanthosis',
       'hyperkeratosis', 'parakeratosis', 'clubbing_of_the_rete_ridges',
       'elongation_of_the_rete_ridges',
       'thinning_of_the_suprapapillary_epidermis', 'spongiform_pustule',
       'munro_microabcess', 'focal_hypergranulosis',
       'disappearance_of_the_granular_layer',
       'vacuolisation_and_damage_of_basal_layer', 'spongiosis',
       'saw-tooth_appearance_of_retes', 'follicular_horn_plug',
       'perifollicular_parakeratosis', 'inflammatory_monoluclear_inflitrate',
       'band-like_infiltrate', 'age', 'class']
dataset.columns=col_name

In [5]:
dataset.head()

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,age,class
0,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
1,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
2,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
3,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3
4,2,3,2,0,0,0,0,0,0,0,...,0,0,2,0,0,0,1,0,41,2


#### Split the dataset
Split the dataset into training, validation and test sets.

First of all, we will get an insight of the information of the dataset with column name added.

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   erythema                                  365 non-null    int64 
 1   scaling                                   365 non-null    int64 
 2   definite_borders                          365 non-null    int64 
 3   itching                                   365 non-null    int64 
 4   koebner_phenomenon                        365 non-null    int64 
 5   polygonal_papules                         365 non-null    int64 
 6   follicular_papules                        365 non-null    int64 
 7   oral_mucosal_involvement                  365 non-null    int64 
 8   knee_and_elbow_involvement                365 non-null    int64 
 9   scalp_involvement                         365 non-null    int64 
 10  family_history                            365 non-

Drop the rows that contains unknown value from the dataset.

In this case, we have some "?" in the column "age".

Hence, we drop the respective rows.

In [7]:
dataset.loc[dataset['age'] == '?']

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,age,class
32,2,2,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,?,1
33,2,1,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,?,4
34,2,2,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,?,2
35,2,1,2,3,2,3,0,2,0,0,...,0,2,0,2,0,0,0,3,?,3
261,2,1,0,2,0,0,0,0,0,0,...,0,0,2,0,0,0,3,0,?,5
262,1,1,1,3,0,0,0,0,0,0,...,0,0,3,0,0,0,2,0,?,5
263,1,1,0,2,0,0,0,0,1,0,...,0,0,2,0,0,0,3,0,?,5
264,1,1,0,3,0,0,0,0,0,0,...,0,0,2,0,0,0,3,0,?,5


In [8]:
dataset.drop(dataset.loc[dataset['age']=='?'].index, inplace=True)

Check again if there any unknown data in the dataset

In [9]:
dataset.loc[dataset['age'] == '?']

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,age,class


In [10]:
dataset

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,age,class
0,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
1,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
2,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
3,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3
4,2,3,2,0,0,0,0,0,0,0,...,0,0,2,0,0,0,1,0,41,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,2,0,25,4
361,3,2,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,2,0,36,4
362,3,2,2,2,3,2,0,2,0,0,...,0,3,0,3,0,0,2,3,28,3
363,2,1,3,1,2,3,0,2,0,0,...,0,2,0,1,0,0,2,3,50,3


After dropping the rows with unknown age, we have 357 rows left.

#### Data splitting

In [11]:
X = dataset.drop(columns = ['class']).copy() #features
y = dataset['class'] #target

In the first step we will split the data in training and remaining dataset

In [12]:
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.7)

Now since we want the valid and test size to be equal (15% each of overall data). 
We have to define valid_size=0.5 (that is 50% of remaining data).

In [13]:
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(249, 34)
(249,)
(54, 34)
(54,)
(54, 34)
(54,)


(None, None)

#### Data preprocessing

Since KNN works based on the distance between data points, it’s important that we standardize the data before training the model. We need to standardize the data first by using StandardScale. The standardized dataset will be saved in the variables "X_train_transform", "X_valid_transform" and "X_test_transform".

The purpose of standardize data for KNN is to gives all features the same influence on the distance metric.

In [14]:
scaler = MinMaxScaler()
X_train_transform = scaler.fit_transform(X_train)
X_valid_transform = scaler.fit_transform(X_valid)
X_test_transform = scaler.fit_transform(X_test)

In [15]:
X_train_transform

array([[0.66666667, 0.        , 0.33333333, ..., 1.        , 0.        ,
        0.82352941],
       [1.        , 0.66666667, 0.33333333, ..., 0.33333333, 0.        ,
        0.63235294],
       [0.33333333, 0.33333333, 0.66666667, ..., 0.66666667, 1.        ,
        0.63235294],
       ...,
       [0.66666667, 0.33333333, 0.66666667, ..., 0.33333333, 0.        ,
        0.14705882],
       [1.        , 0.66666667, 0.66666667, ..., 0.66666667, 0.        ,
        0.04411765],
       [0.66666667, 0.66666667, 1.        , ..., 1.        , 1.        ,
        0.29411765]])

In [16]:
X_valid_transform

array([[0.5       , 0.33333333, 0.        , ..., 0.33333333, 0.        ,
        1.        ],
       [0.5       , 0.66666667, 0.66666667, ..., 0.66666667, 0.        ,
        0.59677419],
       [1.        , 0.66666667, 0.66666667, ..., 0.66666667, 0.        ,
        0.83870968],
       ...,
       [0.5       , 0.66666667, 0.66666667, ..., 1.        , 0.        ,
        0.51612903],
       [1.        , 0.66666667, 0.33333333, ..., 0.66666667, 0.        ,
        0.27419355],
       [0.5       , 0.33333333, 0.66666667, ..., 0.66666667, 1.        ,
        0.69354839]])

In [17]:
X_test_transform

array([[0.33333333, 0.66666667, 0.66666667, ..., 0.66666667, 0.        ,
        0.88571429],
       [0.33333333, 0.33333333, 0.33333333, ..., 0.        , 0.66666667,
        0.        ],
       [1.        , 0.33333333, 0.66666667, ..., 0.66666667, 0.        ,
        0.24285714],
       ...,
       [0.66666667, 0.33333333, 0.66666667, ..., 1.        , 0.        ,
        0.88571429],
       [0.33333333, 1.        , 0.33333333, ..., 1.        , 0.        ,
        0.85714286],
       [0.66666667, 1.        , 0.66666667, ..., 0.66666667, 0.        ,
        0.47142857]])

#### Feature Selection
Perform feature selection to select the relevant features. <br>

*For KNN:*
- We will perform feature selection using forward selection strategy and selects the best subset of features based on accuracy.


*For Neural Network:*
- Feature selection is not as important for neural networks as it is for other algorithms. Neural Networks are “smarter” algorithms, they have internal weights that adjust to minimize a cost function. Less important features will be attributed comparatively lower importance with respect to highly predictive variables1.

In [18]:
!pip install mlxtend
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [19]:
svc=SVC()
svc.fit(X_train,y_train)

forward_fs_best=sfs(estimator = svc, k_features = 'best', forward = True, scoring = 'accuracy')
sfs_forward_best=forward_fs_best.fit(X_train,y_train)

The variable 'feature_index' retrieves the indices of the selected features after performing the forward feature selection using the SequentialFeatureSelector object (sfs_forward_best).

In [20]:
feature_index = np.asarray(sfs_forward_best.k_feature_idx_)
sfs_forward_best.k_feature_names_ #Display the selected features

('koebner_phenomenon',
 'follicular_papules',
 'fibrosis_of_the_papillary_dermis',
 'clubbing_of_the_rete_ridges',
 'disappearance_of_the_granular_layer',
 'saw-tooth_appearance_of_retes',
 'perifollicular_parakeratosis')

In forward selection, the first variable selected for an entry into the constructed model is the one with the largest correlation with the dependent variable. In this case, 'scaling' has the largest correlation with the target class. Other features like 'age' and 'itching' are not informative in determining the class of disease.

In [21]:
#To convert from nparray into dataframe:
X_train_transform_f = pd.DataFrame(X_train_transform)
X_valid_transform_f = pd.DataFrame(X_valid_transform)
X_test_transform_f = pd.DataFrame(X_test_transform)

In [22]:
X_train_fs = X_train_transform_f.iloc[:,feature_index]
X_train_fs

,4,6,14,19,25,28,30
0,0.000000,0.0,0.666667,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.666667,0.0,0.000000,0.000000,0.000000,1.000000,0.000000
3,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333
...,...,...,...,...,...,...,...
244,0.333333,0.0,0.000000,0.333333,0.666667,0.000000,0.000000
245,0.666667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
246,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
247,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.666667


In [23]:
X_valid_fs = X_valid_transform_f.iloc[:,feature_index]
X_valid_fs

,4,6,14,19,25,28,30
0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0
1,0.333333,0.0,0.000000,1.000000,0.333333,0.000000,0.0
2,0.666667,0.0,0.000000,0.666667,0.000000,0.000000,0.0
3,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0
4,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
5,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.0
6,0.333333,0.0,0.000000,0.666667,0.000000,0.000000,0.0
7,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.0
8,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
9,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.0


In [24]:
X_test_fs = X_test_transform_f.iloc[:,feature_index]
X_test_fs

,4,6,14,19,25,28,30
0,0.333333,0.000000,0.000000,1.000000,0.666667,0.000000,0.000000
1,0.333333,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,1.000000,0.666667,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.666667,0.666667,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000
6,0.333333,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000
7,0.666667,0.000000,0.000000,0.000000,0.666667,1.000000,0.000000
8,0.666667,0.000000,0.000000,0.666667,1.000000,0.000000,0.000000
9,0.333333,0.000000,0.000000,0.000000,0.666667,0.666667,0.000000


#### Data modeling
Description: We will be building two predictive models to predict our target variable, which are K-Nearest Neighbours (KNN) and Neural Network. 

### 1. K-Nearest Neighbours

First, we train the neural network model with randomly initialized parameters.

In [45]:
KNN= KNeighborsClassifier(n_neighbors=21)  
KNN.fit(X_train_fs, y_train)
predictions = KNN.predict(X_valid_fs)

print('Validation set: \n')
print('Accuracy:', accuracy_score(y_valid, predictions), '\n')
print('Confusion matrix:\n', confusion_matrix(y_valid, predictions), '\n')
print('Precision, recall, F1-score:\n', classification_report(y_valid, predictions))

Validation set: 

Accuracy: 0.8703703703703703 

Confusion matrix:
 [[16  1  0  0  0  0]
 [ 0 11  0  0  0  0]
 [ 0  1 11  3  0  0]
 [ 0  1  0  4  0  0]
 [ 0  1  0  0  4  0]
 [ 0  0  0  0  0  1]] 

Precision, recall, F1-score:
               precision    recall  f1-score   support

           1       1.00      0.94      0.97        17
           2       0.73      1.00      0.85        11
           3       1.00      0.73      0.85        15
           4       0.57      0.80      0.67         5
           5       1.00      0.80      0.89         5
           6       1.00      1.00      1.00         1

    accuracy                           0.87        54
   macro avg       0.88      0.88      0.87        54
weighted avg       0.91      0.87      0.88        54



Then, we find the best parameters of KNN by using GridSearchCV and compare the result with the previous result produced with random parameters to prove the improvements.

In [26]:
knn_params={
    "n_neighbors":range(1,30,2),
    "weights":["uniform", "distance"],
    "metric":["euclidean", "manhattan","minkowski"],
    "algorithm":["auto","ball_tree","kd_tree","brute"],
    "leaf_size":range(1,50,5)
}

#grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
grid_search = GridSearchCV(estimator=KNN, param_grid=knn_params, n_jobs=-2, cv=cv, scoring="accuracy", error_score=0)
grid_results = grid_search.fit(X_train_fs, y_train)

#best model
final_model = KNN.set_params(**grid_results.best_params_)

In [27]:
final_model.fit(X_train_fs, y_train)
pred = final_model.predict(X_valid_fs)

print('Validation set: \n')
print('Accuracy:', accuracy_score(y_valid, pred), '\n')
print('Confusion matrix:\n', confusion_matrix(y_valid, pred), '\n')
print('Precision, recall, F1-score:\n', classification_report(y_valid, pred))

Validation set: 

Accuracy: 0.9259259259259259 

Confusion matrix:
 [[17  0  0  0  0  0]
 [ 0 11  0  0  0  0]
 [ 0  1 11  3  0  0]
 [ 0  0  0  5  0  0]
 [ 0  0  0  0  5  0]
 [ 0  0  0  0  0  1]] 

Precision, recall, F1-score:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        17
           2       0.92      1.00      0.96        11
           3       1.00      0.73      0.85        15
           4       0.62      1.00      0.77         5
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         1

    accuracy                           0.93        54
   macro avg       0.92      0.96      0.93        54
weighted avg       0.95      0.93      0.93        54



### 2. Neural Network

In [28]:
def create_model(units=100):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(34,)),
    tf.keras.layers.Dense(units, activation='relu', dtype='float64', kernel_regularizer='l1'),
    tf.keras.layers.Dense(units, activation='relu', dtype='float64', kernel_regularizer='l1'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer='l1')
    ])
    model.compile(optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])    
    return model

First, we train the neural network model with randomly initialized parameters.

In [51]:
randomP_model = create_model(32) 
randomP_model.fit(X_train_transform, np.delete(to_categorical(y_train),0,1), validation_data=(X_valid_transform, np.delete(to_categorical(y_valid),0,1)), batch_size=32, epochs=10)

Epoch 1/10
8/8 [==============================] - 1s 52ms/step - loss: 5.6836 - accuracy: 0.1888 - val_loss: 5.3606 - val_accuracy: 0.1667
Epoch 2/10
8/8 [==============================] - 0s 8ms/step - loss: 4.9893 - accuracy: 0.4498 - val_loss: 5.1801 - val_accuracy: 0.3519
Epoch 3/10
8/8 [==============================] - 0s 8ms/step - loss: 4.5454 - accuracy: 0.6426 - val_loss: 5.0396 - val_accuracy: 0.3889
Epoch 4/10
8/8 [==============================] - 0s 7ms/step - loss: 4.2701 - accuracy: 0.7590 - val_loss: 4.9069 - val_accuracy: 0.6111
Epoch 5/10
8/8 [==============================] - 0s 8ms/step - loss: 4.0431 - accuracy: 0.8514 - val_loss: 4.7763 - val_accuracy: 0.7222
Epoch 6/10
8/8 [==============================] - 0s 7ms/step - loss: 3.8763 - accuracy: 0.8916 - val_loss: 4.6480 - val_accuracy: 0.7778
Epoch 7/10
8/8 [==============================] - 0s 8ms/step - loss: 3.7486 - accuracy: 0.9116 - val_loss: 4.5250 - val_accuracy: 0.7963
Epoch 8/10
8/8 [=================

In [52]:
y_pred = randomP_model.predict(X_valid_transform)
y_pred = 1 + np.argmax(y_pred, axis=1)

print('Validation set: \n')
print('Accuracy:',accuracy_score(y_valid, y_pred), '\n')
print('Confusion matrix:\n',confusion_matrix(y_valid, y_pred), '\n')
print('Precision, recall, F1-score:\n',classification_report(y_valid, y_pred))

2/2 [==============================] - 0s 7ms/step
Validation set: 

Accuracy: 0.8518518518518519 

Confusion matrix:
 [[17  0  0  0  0  0]
 [ 0  7  0  4  0  0]
 [ 0  0 15  0  0  0]
 [ 1  2  0  2  0  0]
 [ 1  0  0  0  4  0]
 [ 0  0  0  0  0  1]] 

Precision, recall, F1-score:
               precision    recall  f1-score   support

           1       0.89      1.00      0.94        17
           2       0.78      0.64      0.70        11
           3       1.00      1.00      1.00        15
           4       0.33      0.40      0.36         5
           5       1.00      0.80      0.89         5
           6       1.00      1.00      1.00         1

    accuracy                           0.85        54
   macro avg       0.83      0.81      0.82        54
weighted avg       0.86      0.85      0.85        54



Next, we find the best parameters of Neural Network by using GridSearchCV and compare the result with the previous result produced with random parameters to prove the improvements.

In [31]:
param_grid = {
    'units': [64, 100, 128],
    'batch_size': [32, 64, 100],
    'epochs': [50, 100, 150]  
}

In [32]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=64, verbose=0)

In [33]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy')

In [34]:
grid_search.fit(X_train_transform,y_train)

2/2 [==============================] - 0s 2ms/step


GridSearchCV(estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000021C602F7790>,
             param_grid={'batch_size': [32, 64, 100], 'epochs': [50, 100, 150],
                         'units': [64, 100, 128]},
             scoring='accuracy')

In [35]:
print("Best Hyperparameters: ", grid_search.best_params_)

Best Hyperparameters:  {'batch_size': 32, 'epochs': 100, 'units': 64}


In [36]:
y_train_c = np.delete(to_categorical(y_train),0,1)
y_val_c = np.delete(to_categorical(y_valid),0,1)

In [37]:
best_model = create_model(grid_search.best_params_['units']) 
best_model.fit(X_train_transform, y_train_c ,
validation_data=(X_valid_transform, y_val_c),
batch_size=grid_search.best_params_['batch_size'], epochs=grid_search.best_params_['epochs'])

Epoch 1/100
8/8 [==============================] - 1s 45ms/step - loss: 9.4335 - accuracy: 0.3012 - val_loss: 9.0938 - val_accuracy: 0.2778
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 8.2406 - accuracy: 0.7028 - val_loss: 8.7016 - val_accuracy: 0.7963
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 7.6575 - accuracy: 0.8635 - val_loss: 8.3547 - val_accuracy: 0.8889
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 7.2646 - accuracy: 0.9197 - val_loss: 8.0399 - val_accuracy: 0.9259
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 6.9547 - accuracy: 0.9277 - val_loss: 7.7452 - val_accuracy: 0.9259
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 6.6587 - accuracy: 0.9478 - val_loss: 7.4638 - val_accuracy: 0.9259
Epoch 7/100
8/8 [==============================] - 0s 6ms/step - loss: 6.3758 - accuracy: 0.9598 - val_loss: 7.1911 - val_accuracy: 0.9259
Epoch 8/100
8/8 [=========

In [38]:
y_pred = best_model.predict(X_valid_transform)
y_pred = 1 + np.argmax(y_pred, axis=1)

print('Validation set: \n')
print('Accuracy:',accuracy_score(y_valid, y_pred), '\n')
print('Confusion matrix:\n',confusion_matrix(y_valid, y_pred), '\n')
print('Precision, recall, F1-score:\n',classification_report(y_valid, y_pred))

2/2 [==============================] - 0s 3ms/step
Validation set: 

Accuracy: 1.0 

Confusion matrix:
 [[17  0  0  0  0  0]
 [ 0 11  0  0  0  0]
 [ 0  0 15  0  0  0]
 [ 0  0  0  5  0  0]
 [ 0  0  0  0  5  0]
 [ 0  0  0  0  0  1]] 

Precision, recall, F1-score:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        17
           2       1.00      1.00      1.00        11
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         1

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54



#### Evaluate the models
Perform a comparison between the predictive models. <br>
Report the accuracy, recall, precision and F1-score measures as well as the confusion matrix.

### K-Nearest Neighbors

In [39]:
y_predict1 = final_model.predict(X_test_fs)

print('Accuracy:', accuracy_score(y_test,y_predict1), '\n')
print('Confusion matrix:\n', confusion_matrix(y_test,y_predict1), '\n')
print('Precision, recall, F1-score:\n', classification_report(y_test,y_predict1))

Accuracy: 0.9259259259259259 

Confusion matrix:
 [[19  3  0  0  0  0]
 [ 0  5  0  0  0  0]
 [ 0  1 10  0  0  0]
 [ 0  0  0  6  0  0]
 [ 0  0  0  0  6  0]
 [ 0  0  0  0  0  4]] 

Precision, recall, F1-score:
               precision    recall  f1-score   support

           1       1.00      0.86      0.93        22
           2       0.56      1.00      0.71         5
           3       1.00      0.91      0.95        11
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         4

    accuracy                           0.93        54
   macro avg       0.93      0.96      0.93        54
weighted avg       0.96      0.93      0.93        54



### Neural Network

In [40]:
y_pred = best_model.predict(X_test_transform)
y_pred = np.argmax(y_pred, axis=1) + 1

2/2 [==============================] - 0s 2ms/step


In [41]:
print('Accuracy:',accuracy_score(y_test, y_pred), '\n')
print('Confusion matrix:\n', confusion_matrix(y_test, y_pred), '\n')
print('Precision, recall, F1-score:\n',classification_report(y_test, y_pred))

Accuracy: 1.0 

Confusion matrix:
 [[22  0  0  0  0  0]
 [ 0  5  0  0  0  0]
 [ 0  0 11  0  0  0]
 [ 0  0  0  6  0  0]
 [ 0  0  0  0  6  0]
 [ 0  0  0  0  0  4]] 

Precision, recall, F1-score:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        22
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00        11
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         4

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

